In [20]:
from dotenv import load_dotenv
import chromadb
import openai
import os

load_dotenv()
openai_base_url = os.getenv('OPENAI_BASE_URL')
openai_api_key = os.getenv("LLM_TOKEN")
CHROMA_DATA_PATH = "../chroma_data/"
collection_db="article_comments"

#Initialize a client
client = openai.OpenAI(api_key=openai_api_key, base_url=openai_base_url)

chroma_client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)


In [21]:
def ask_question_openai(question:str, collection: chromadb.PersistentClient() = collection_db, n_docs:int = 30, filters: dict ={}) -> str:
    
    # Find close documents in chromadb
    collection = chroma_client.get_collection(collection)
    results = collection.query(
       query_texts=[question],
       n_results=n_docs,
       where=filters
    )

    # Collect the results in a context
    context = "\n".join([r for r in results['documents'][0]])
    
    prompt = f"""
    Answer the following question: {question}.  
    Refer only to the following information when answering: {context}.
    Provide at least six paragraphs in summary if it is possible given the information provided.
    Begin your answer with 'Based on the responses from selected NY Times readers', and try to give a sense of majority and minority opinions on the topic, but only if there is an identifiable majority opinion.
    If there is not enough information provided to give a summarized opinion, indicate that this is the case.
    """
    
    chat_completion = client.chat.completions.create(
        messages=[{
                "role": "user",
                "content": prompt,
        }],
        model="gpt-4",
    )
    
    print(chat_completion.choices[0].message.content)

In [22]:
ask_question_openai("What do readers think about the music of Taylor Swift?")

Based on the responses from selected NY Times readers, there is quite a considerable variation in the opinions about the music of Taylor Swift. Some readers appreciated Taylor's music and look forward to her new albums, expressing enthusiasm for her particular brand of songs, and recognizing her struggles in the public eye. Attention was also drawn to the popularity of her music across various demographics, inferring that her music appeals to a broad fan base. These fans praised her strength and resilience after her sexual assault incident, commending her for becoming a role model for other women.

However, there was a significant portion of dissenting voices as well. These readers criticized her for falling into the formulaic mold of pop music, suggesting that her songs are crafted by teams rather than herself, hence lacking authenticity. One reader even dismissed her music as 'garbage,' arguing that it was merely a product of the controlled music industry where quality is compromised